In [1]:
import time
import torch
import numpy as np
import deepxde as dde
import pandas as pd
from sklearn.preprocessing import StandardScaler  
from deepxde.nn.pytorch.fnn import FNN
from deepxde.data.data import Tuple
from sklearn.preprocessing import MinMaxScaler

Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


NVIDIA GeForce RTX 4090 D


In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda:0') 		
print(torch.cuda.get_device_name(0))

In [2]:
def auto_move_last_row(data):
    num_rows = data.shape[0]
    if num_rows == 0:
        print("The array is empty.")
        return data
    last_row = data[-1]
    if num_rows == 1:
        return data
    data = np.delete(data, -1, axis=0)
    data = np.insert(data, 0, last_row, axis=0)
    return data
def auto_move_first_row(data):
    num_rows = data.shape[0]
    if num_rows == 0:
        print("The array is empty.")
        return data
    first_row = data[0]
    if num_rows == 1:
        return data
    data = np.delete(data, 0, axis=0)
    data = np.vstack((data, first_row))
    return data

In [3]:
def elementwise_l2_error(matrix_true, matrix_pred):
    diff = matrix_true - matrix_pred
    squared_diff = np.square(diff)
    l2_error_matrix = np.sqrt(squared_diff)
    return l2_error_matrix

In [4]:
def data_transorm(p):  
    array = p.values[0::, 0::] 
    empty = []
    for i in range(len(p)):
        lst = array[i][0].split(" ")
        new_lst = list(filter(lambda x: x.strip(), lst))
        new_lst2 = [float(x) for x in new_lst]
        new_lst2 = np.array(new_lst2, dtype='float32')
        empty.append(new_lst2)
    empty = np.array(empty, dtype='float32')
    scaler = StandardScaler()
    empty = scaler.fit_transform(empty)   
    return empty, scaler
def data_transorm_1(p): 
    array = p.values[0::, 0::]  
    empty = []
    for i in range(len(p)):
        lst = array[i][0].split(" ")
        new_lst = list(filter(lambda x: x.strip(), lst))
        new_lst2 = [float(x) for x in new_lst]
        new_lst2 = np.array(new_lst2, dtype='float32')
        empty.append(new_lst2)
    empty = np.array(empty, dtype='float32')
    return empty

In [5]:
class DatasetsConstruct_IC:
    def for_train(self, num, start):
        self.num = num
        self.start = start
        for i in range(self.num):
            num1 = str(self.start + 20 * i)
            E_train = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num1 + "\\" + "E_train.csv",encoding='gbk', header=None)
            E_train = data_transorm_1(E_train)
            E_train_IC = E_train[0].reshape(-1,1)

            E_test = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num1 + "\\" + "E_test.csv",encoding='gbk', header=None)
            E_test = data_transorm_1(E_test)
            E_test_IC = E_test[0].reshape(-1,1)

            num = np.linspace(start=1, stop=200, num=200, dtype='float32').reshape(-1, 1)
            voltage = np.full(200, self.start + 20 * i, dtype='float32').reshape(-1, 1)
            input_train = np.hstack((num, voltage))

            print("已读取：", "V_" + num1)
            if i == 0:
                temp_num = num
                temp_voltage = voltage
                temp_E_train = E_train_IC
                temp_E_test = E_test_IC
                temp_input_train = input_train
            else:
                temp_num = np.concatenate((temp_num, num), axis=0)
                temp_voltage = np.concatenate((temp_voltage, voltage), axis=0)
                temp_E_train = np.concatenate((temp_E_train, E_train_IC), axis=0)
                temp_E_test = np.concatenate((temp_E_test, E_test_IC), axis=0)
                temp_input_train = np.concatenate((temp_input_train, input_train), axis=0)
        x_scaler=MinMaxScaler(feature_range=(0,1))
        y_scaler=MinMaxScaler(feature_range=(0,1))
        self.x_scaler = x_scaler
        self.y_scaler = y_scaler
        num = temp_num
        voltage = temp_voltage
        E_train = temp_E_train
        E_train=x_scaler.fit_transform(E_train)
        E_test = temp_E_test
        E_test=x_scaler.transform(E_test)
        input_train = temp_input_train
        input_train=y_scaler.fit_transform(input_train)

        data_train = Tuple(input_train, E_train, input_train, E_test)

        U_col = np.size(input_train, 1)  # 计算 X 的列数
        branch_net_input = U_col

        return data_train

    def restore(self, ic):
        ic = self.x_scaler.inverse_transform(ic)
        return ic
dataset_IC = DatasetsConstruct_IC()
data_train_IC= dataset_IC.for_train(num=11,start=400)
net_IC = FNN(
    [2,300, 300,300, 300,1],
    "gelu",
    "Glorot normal"
)
model_IC = dde.Model(data_train_IC, net_IC)
model_IC.compile("adam", lr=0.0005, metrics=["mean l2 relative error"])
model_IC.restore('H:\\DNNISS4RF\\model\\IC.pt')

已读取： V_400
已读取： V_420
已读取： V_440
已读取： V_460
已读取： V_480
已读取： V_500
已读取： V_520
已读取： V_540
已读取： V_560
已读取： V_580
已读取： V_600
Compiling model...
'compile' took 0.547743 s



In [6]:
def DatasetsConstruct_4train(num, start):
    for i in range(num):
        num1 = str(start + 20 * i)

        Ne_train = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num1 + "\\" + "Ne_train.csv",encoding='gbk', header=None)
        Ne_train, temp = data_transorm(Ne_train)

        Ne_test = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num1 + "\\" + "Ne_test.csv",encoding='gbk', header=None)
        Ne_test, temp = data_transorm(Ne_test)

        Te_train = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num1 + "\\" + "Te_train.csv",encoding='gbk', header=None)
        Te_train, temp = data_transorm(Te_train)
        
        Te_test = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num1 + "\\" + "Te_test.csv",encoding='gbk', header=None)
        Te_test, temp = data_transorm(Te_test)

        Nion_train = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num1 + "\\" + "Nion_train.csv",encoding='gbk', header=None)
        Nion_train, temp = data_transorm(Nion_train)

        Nion_test = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num1 + "\\" + "Nion_test.csv",encoding='gbk', header=None)
        Nion_test, temp = data_transorm(Nion_test)

        Je_train = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num1 + "\\" + "Je_train.csv",encoding='gbk', header=None)
        Je_train, temp = data_transorm(Je_train)

        Je_test = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num1 + "\\" + "Je_test.csv",encoding='gbk', header=None)
        Je_test, temp = data_transorm(Je_test)

        Ji_train = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num1 + "\\" + "Ji_train.csv",encoding='gbk', header=None)
        Ji_train, temp = data_transorm(Ji_train)
        
        Ji_test = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num1 + "\\" + "Ji_test.csv",encoding='gbk', header=None)
        Ji_test, temp = data_transorm(Ji_test)

        E_train = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num1 + "\\" + "E_train.csv",encoding='gbk', header=None)
        E_train, temp = data_transorm(E_train)

        E_test = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num1 + "\\" + "E_test.csv",encoding='gbk', header=None)
        E_test, temp = data_transorm(E_test)

        print("已读取：", "V_" + num1)
        if i == 0:
            temp_Ne_train = Ne_train
            temp_Ne_test = Ne_test
            temp_Te_train = Te_train
            temp_Te_test = Te_test
            temp_Nion_train = Nion_train
            temp_Nion_test = Nion_test
            temp_Je_train = Je_train
            temp_Je_test = Je_test
            temp_Ji_train = Ji_train
            temp_Ji_test = Ji_test
            temp_E_train = E_train
            temp_E_test = E_test

        else:
            temp_Ne_train = np.concatenate((temp_Ne_train, Ne_train), axis=0)
            temp_Ne_test = np.concatenate((temp_Ne_test, Ne_test), axis=0)
            temp_Nion_train = np.concatenate((temp_Nion_train, Nion_train), axis=0)
            temp_Nion_test = np.concatenate((temp_Nion_test, Nion_test), axis=0)
            temp_Te_train = np.concatenate((temp_Te_train, Te_train), axis=0)
            temp_Te_test = np.concatenate((temp_Te_test, Te_test), axis=0)
            temp_Je_train = np.concatenate((temp_Je_train, Je_train), axis=0)
            temp_Je_test = np.concatenate((temp_Je_test, Je_test), axis=0)
            temp_Ji_train = np.concatenate((temp_Ji_train, Ji_train), axis=0)
            temp_Ji_test = np.concatenate((temp_Ji_test, Ji_test), axis=0)
            temp_E_train = np.concatenate((temp_E_train, E_train), axis=0)
            temp_E_test = np.concatenate((temp_E_test, E_test), axis=0)

    Ne_train = temp_Ne_train
    Ne_test = temp_Ne_test
    Te_train = temp_Te_train
    Te_test = temp_Te_test
    Nion_train = temp_Nion_train
    Nion_test = temp_Nion_test
    Je_train = temp_Je_train
    Je_test = temp_Je_test
    Ji_train = temp_Ji_train
    Ji_test = temp_Ji_test
    E_train = temp_E_train
    E_test = temp_E_test

    dataset_Ne2Te = Tuple(Ne_train, Te_train, Ne_test, Te_test)

    dataset_Ji2Ni = Tuple(Ji_train, Nion_train, Ji_test, Nion_test)

    dataset_je2Ne = Tuple(Je_train, Ne_train, Je_test, Ne_test)

    dataset_E2ji = Tuple(E_train, Ji_train, E_test, Ji_test)

    dataset_E2je = Tuple(E_train, Je_train, E_test, Je_test)

    dataset_Nei2E = Tuple(Ne_train, E_train, Ne_test, E_test)

    return dataset_Ne2Te, dataset_Ji2Ni, dataset_je2Ne, dataset_E2ji, dataset_E2je, dataset_Nei2E

data_train_Ne2Te, data_train_Ji2Ni, data_train_je2Ne, data_train_E2ji, data_train_E2je, data_train_Nei2E = DatasetsConstruct_4train(num=11,start=400)

已读取： V_400
已读取： V_420
已读取： V_440
已读取： V_460
已读取： V_480
已读取： V_500
已读取： V_520
已读取： V_540
已读取： V_560
已读取： V_580
已读取： V_600


In [8]:
net_Nei2E = FNN(
    [200,300, 400,400, 300,200],
    "gelu",
    "Glorot normal"
)
model_Nei2E = dde.Model(data_train_Nei2E, net_Nei2E)
model_Nei2E.compile("adam", lr=0.0005, metrics=["mean l2 relative error"])
model_Nei2E.restore('H:\\DNNISS4RF\\model\\Nei2E.pt')

net_E2je = FNN(
    [200,300, 400,400, 300,200],
    "gelu",
    "Glorot normal"
)
model_E2je = dde.Model(data_train_E2je, net_E2je)
model_E2je.compile("adam", lr=0.0005, metrics=["mean l2 relative error"])
model_E2je.restore('H:\\DNNISS4RF\\model\\E2je.pt')

net_E2ji = FNN(
    [200,300, 400,400, 300,200],
    "gelu",
    "Glorot normal"
)
model_E2ji = dde.Model(data_train_E2ji, net_E2ji)
model_E2ji.compile("adam", lr=0.0005, metrics=["mean l2 relative error"])
model_E2ji.restore('H:\\DNNISS4RF\\model\\E2ji.pt')

net_je2Ne = FNN(
    [200,300, 400,400, 300,200],
    "gelu",
    "Glorot normal"
)
model_je2Ne = dde.Model(data_train_je2Ne, net_je2Ne)
model_je2Ne.compile("adam", lr=0.0005, metrics=["mean l2 relative error"])
model_je2Ne.restore('H:\\DNNISS4RF\\model\\Je2Ne.pt')

net_Ji2Ni = FNN(
    [200,300, 400,400, 300,200],
    "gelu",
    "Glorot normal"
)
model_Ji2Ni = dde.Model(data_train_Ji2Ni, net_Ji2Ni)
model_Ji2Ni.compile("adam", lr=0.0005, metrics=["mean l2 relative error"])
model_Ji2Ni.restore('H:\\DNNISS4RF\\model\\Ji2Ni.pt')

net_Ne2Te = FNN(
    [200,300, 400,400, 300,200],
    "gelu",
    "Glorot normal"
)
model_Ne2Te = dde.Model(data_train_Ne2Te, net_Ne2Te)
model_Ne2Te.compile("adam", lr=0.0005, metrics=["mean l2 relative error"])
model_Ne2Te.restore('H:\\DNNISS4RF\\model\\Ne2Te.pt')

Compiling model...
'compile' took 0.000353 s

Compiling model...
'compile' took 0.000280 s

Compiling model...
'compile' took 0.000237 s

Compiling model...
'compile' took 0.000243 s

Compiling model...
'compile' took 0.000271 s

Compiling model...
'compile' took 0.000236 s



In [9]:
def DNNISS(voltage):
    num2 = str(voltage)

    Ne_verify = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num2 + "\\" + "Ne_train.csv",encoding='gbk', header=None)
    Ne_verify1, Ne_scaler  = data_transorm(Ne_verify)
    Ne_verify = Ne_scaler.inverse_transform(Ne_verify1)

    Nion_verify = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num2 + "\\" + "Nion_train.csv",encoding='gbk', header=None)
    Nion_verify1, Nion_scaler  = data_transorm(Nion_verify)
    Nion_verify = Nion_scaler.inverse_transform(Nion_verify1)
    
    E_verify = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num2 + "\\" + "E_train.csv",encoding='gbk', header=None)
    E_verify1, E_scaler  = data_transorm(E_verify)
    E_verify = E_scaler.inverse_transform(E_verify1)
    
    Je_verify = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num2 + "\\" + "Je_train.csv",encoding='gbk', header=None)
    Je_verify1, Je_scaler = data_transorm(Je_verify)
    Je_verify = Je_scaler.inverse_transform(Je_verify1)
    
    Ji_verify = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num2 + "\\" + "Ji_train.csv",encoding='gbk', header=None)
    Ji_verify1, Ji_scaler = data_transorm(Ji_verify)
    Ji_verify = Ji_scaler.inverse_transform(Ji_verify1)

    Te_verify = pd.read_csv("H:\\DNNISS4RF\\datasets\\" + "V_" + num2 + "\\" + "Te_train.csv",encoding='gbk', header=None)
    Te_verify1, Te_scaler = data_transorm(Te_verify)
    Te_verify = Te_scaler.inverse_transform(Te_verify1)

    start_time = time.time()
    E_pred1 = []
    Je_pred1 = []
    Ji_pred1 = []
    Ne_pred1 = []
    Nion_pred1 = []
    Te_pred1 = []

    ic_transform = np.array([[1,400],[200,600]], dtype='float32')
    ic_scaler=MinMaxScaler(feature_range=(0,1))
    ic_transform = ic_scaler.fit_transform(ic_transform)
    num_validate = np.linspace(start=1, stop=200, num=200, dtype='float32').reshape(-1, 1)
    voltage_validate = np.full(200, voltage, dtype='float32').reshape(-1, 1)
    ic_input = np.hstack((num_validate, voltage_validate))
    ic_input = ic_scaler.transform(ic_input)
    ic_pred = model_IC.predict(ic_input)
    ic_pred = dataset_IC.restore(ic_pred)
    E_ic = E_scaler.transform(ic_pred.reshape(1, -1))
    
    for i in range(np.size(Ne_verify1, 0)):
        E_input = E_ic.astype(np.float32)
        Je_pred = model_E2je.predict(E_input)
        Ji_pred = model_E2ji.predict(E_input)
        Je_pred1.append(Je_pred)
        Ji_pred1.append(Ji_pred)
        Je_input = Je_pred.astype(np.float32)
        Ji_input = Ji_pred.astype(np.float32)
        Ne_pred = model_je2Ne.predict(Je_input)
        Ni_pred = model_Ji2Ni.predict(Ji_input)
        Ne_pred1.append(Ne_pred)
        Nion_pred1.append(Ni_pred)
        Te_input = Ne_pred.astype(np.float32)
        Te_pred = model_Ne2Te.predict(Te_input)
        Te_pred1.append(Te_pred)
        Nei_input = Ne_pred
        E_pred = model_Nei2E.predict(Nei_input)
        E_pred1.append(E_pred)  
        E_ic = E_pred
 
    end_time = time.time()
    time_difference = end_time - start_time
    print('预测耗时：',time_difference,'s')
     
    E_pred1 = np.array(E_pred1)
    E_pred1 = np.squeeze(E_pred1)
    E_pred1 = auto_move_last_row(E_pred1)
    E_pred = E_scaler.inverse_transform(E_pred1)
    
    Je_pred1 = np.array(Je_pred1)
    Je_pred1 = np.squeeze(Je_pred1)
    Je_pred = Je_scaler.inverse_transform(Je_pred1)
    
    Ji_pred1 = np.array(Ji_pred1)
    Ji_pred1 = np.squeeze(Ji_pred1)
    Ji_pred = Ji_scaler.inverse_transform(Ji_pred1)
    
    Ne_pred1 = np.array(Ne_pred1)
    Ne_pred1 = np.squeeze(Ne_pred1)
    Ne_pred = Ne_scaler.inverse_transform(Ne_pred1)
    
    Nion_pred1 = np.array(Nion_pred1)
    Nion_pred1 = np.squeeze(Nion_pred1)
    Nion_pred = Nion_scaler.inverse_transform(Nion_pred1)

    Te_pred1 = np.array(Te_pred1)
    Te_pred1 = np.squeeze(Te_pred1)
    Te_pred = Te_scaler.inverse_transform(Te_pred1)
    
    df1 = pd.DataFrame(data=Ne_pred)
    df1.to_csv('H:\\DNNISS4RF\\pred\\'+'V_'+num2+' Ne_pred.csv', index=False, header=False,mode='w')
    df2 = pd.DataFrame(data=E_pred)
    df2.to_csv('H:\\DNNISS4RF\\pred\\'+'V_'+num2+' E_pred.csv', index=False, header=False,mode='w')
    df3 = pd.DataFrame(data=Je_pred)
    df3.to_csv('H:\\DNNISS4RF\\pred\\'+'V_'+num2+' Je_pred.csv', index=False, header=False,mode='w')
    df4 = pd.DataFrame(data=Ji_pred)
    df4.to_csv('H:\\DNNISS4RF\\pred\\'+'V_'+num2+' Ji_pred.csv', index=False, header=False,mode='w')
    df5 = pd.DataFrame(data=Nion_pred)
    df5.to_csv('H:\\DNNISS4RF\\pred\\'+'V_'+num2+' Nion_pred.csv', index=False, header=False,mode='w')
    df6 = pd.DataFrame(data=Te_pred)
    df6.to_csv('H:\\DNNISS4RF\\pred\\'+'V_'+num2+' Te_pred.csv', index=False, header=False,mode='w')

    #error
    Ne_error = dde.metrics.l2_relative_error(Ne_verify, Ne_pred)
    Ni_error = dde.metrics.l2_relative_error(Nion_verify, Nion_pred)
    E_error = dde.metrics.l2_relative_error(E_verify, E_pred)
    Je_error = dde.metrics.l2_relative_error(Je_verify, Je_pred)
    Ji_error = dde.metrics.l2_relative_error(Ji_verify, Ji_pred)
    Te_error = dde.metrics.l2_relative_error(Te_verify, Te_pred)

    print('Ne_error',Ne_error)
    print('Ni_error',Ni_error)
    print('E_error',E_error)
    print('Je_error',Je_error)
    print('Ji_error',Ji_error)
    print('Te_error',Te_error)

In [10]:
DNNISS(500)

预测耗时： 0.5590000152587891 s
Ne_error 0.00031201637
Ni_error 1.1220948e-05
E_error 0.0015289898
Je_error 0.0
Ji_error 0.00168442
Te_error 0.00060871453
